Group: 008-31

Members: Matthew Choi, Elizaveta Lazareva, Bianca Swartz, Vicky Yang

# Predicting Newsletter Subscription Using Player Demographics and Gameplay Behavior

## Introduction

### Background

In this project, we investigate a real dataset collected by a research group in the Computer Science department at UBC. The team operates a custom Minecraft research server where player activity is recorded. The goal of the project is to help the research group understand player behaviour, manage resources, and target recruitment efforts more effectively. 

The server collects data about each player's demographic information as well as their in-game behaviour. Understanding how players interact with the server has practical importance. It affects decisions such as when to allocate server resources, what types of players are most valuable to recruit, and what characteristics are associated with certain behaviors such as subscribing to project newsletters.

### Question

In this report, we address the following specific question:

**Can a player's age, gender, experience level, total playtime, number of sessions, and average session duration be used to predict whether they subscribe to the newsletter?**

The aim of the question is to determine which features of a player (such as age, gender, experience, number of sessions, total playtime, and other behavioural summaries) are useful for predicting whether the player subscribed to the newsletter. Being able to predict this provides insight into which groups of players show more engagement with the project, which can help guide future outreach and recruitment strategies.

### Dataset Description

Two datasets are used in this project:

1. **players.csv**
    196 observations. Variables include:
    - Age (numeric)
    - gender (categorical)
    - experience (categorical)
    - played_hours (numeric)
    - subscribe (True or False)
    - hashedEmail (join key)
    - name (not used for analysis)|
2. **sessions.csv**  
    1535 observations. Variables include:
    - hashedEmail (foreign key)
    - original_start_time
    - original_end_time
    - start_time, end_time

### Plan for Data Usage
1. Compute session summaries: total sessions and average duration.
2. Join summaries with players.csv.
3. Use cleaned variables as predictors for kNN classification.

These processed variables will be used as predictors in a classification model that attempts to predict newsletter subscription.

